In [44]:
# Khai báo các thư viện cần thiết
import os
import random
import pickle

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [45]:
# Khai báo tên meta model
while True:
    model_name = input("Enter meta model name: ")
    if model_name.strip() != "":
        break
    else:
        print("Model name cannot be empty. Please enter a valid name.")
os.makedirs(model_name, exist_ok=True)

In [46]:
# Lớp TrainingData 
# Lớp này dùng để lưu trữ dữ liệu huấn luyện cho mô hình

class TrainingData:
    def __init__(self):
        """
        Khởi tạo lớp TrainingData với dữ liệu và nhãn.
        
        :param data: Dữ liệu huấn luyện (danh sách các mẫu).
        :param labels: Nhãn tương ứng với dữ liệu (danh sách các nhãn).
        """
        self.data = []
    
    def add(self, sample):
        """
        Thêm một mẫu vào dữ liệu huấn luyện.
        
        :param sample: Mẫu dữ liệu cần thêm.
        """
        self.data.append(sample)

    def sample(self, batch_size):
        """
        Lấy một mẫu ngẫu nhiên từ dữ liệu huấn luyện với kích thước batch_size.
        
        :param batch_size: Kích thước của mẫu cần lấy.
        :return: Một danh sách chứa các mẫu ngẫu nhiên.
        """
        return random.sample(self.data, batch_size) if len(self.data) >= batch_size else self.data
    
    def __len__(self):
        """
        Trả về số lượng mẫu trong dữ liệu huấn luyện.
        
        :return: Số lượng mẫu trong dữ liệu.
        """
        return len(self.data)

In [47]:
# Nạp dữ liệu huấn luyện vào training_data
training_data = TrainingData()

models_info = []  # Danh sách chứa thông tin mô hình

for model_folder in os.listdir():
    if os.path.exists(model_folder + "/model.pth") and os.path.exists(model_folder + "/model_info.pkl") and os.path.exists(model_folder + "/dataset.pkl"):
        with open(model_folder + "/model_info.pkl", 'rb') as f:
            model_info = pickle.load(f) 
            models_info.append(model_info)
        with open(model_folder + "/dataset.pkl", 'rb') as f:
            dataset = pickle.load(f)
        for data in dataset:
            training_data.add(data)
    else:
        continue
        
print(f"Loaded {len(training_data)} samples from {len(models_info)} classes.")

# Lưu models_info vào file
with open(model_name + "/models_info.pkl", 'wb') as f:
    pickle.dump(models_info, f)
print(f"Saved models_info to {model_name}/models_info.pkl")

Loaded 18786 samples from 5 classes.
Saved models_info to meta_model01/models_info.pkl


In [48]:
# Mạng nơron

class MetaModelNet(nn.Module):
    """
    Mạng nơron cho mô hình MetaModelNet.
    """
    def __init__(self, num_classes):
        super(MetaModelNet, self).__init__()
    
        # Tầng FC cho position
        self.fc_position = nn.Linear(2, 32)
        # Dropout trước tầng FC cuối cùng
        self.dropout = nn.Dropout(0.5)
        # Tầng đầu ra
        self.fc = nn.Linear(32, num_classes)

    def forward(self, position):
        return self.fc(self.dropout(F.relu(self.fc_position(position))))

In [49]:
# Lưu, tải trọng số của mô hình
def save_model(model, filename):
    """
    Lưu trọng số của mô hình vào file.
    
    :param model: Mô hình cần lưu.
    :param filename: Tên file để lưu trọng số.
    """
    torch.save(model, filename)
    print(f"Model saved to {filename}")

def load_model(filename, num_classes, device):
    """
    Tải trọng số của mô hình từ file.
    
    :param model: Mô hình cần tải trọng số.
    :param filename: Tên file chứa trọng số.
    """
    if os.path.exists(filename):
        model = torch.load(filename, map_location=device, weights_only=False)
        print(f"Model loaded from {filename}")
        return model
    else:
        print(f"File {filename} does not exist.")
        return MetaModelNet(num_classes).to(device)

In [50]:
# Khởi tạo các siêu tham số
num_classes = len(models_info)  # Số lớp (số nhãn)

max_episodes = 50000  # Số lượng episode tối đa

batch_size = 32  # Kích thước batch

learning_rate = 0.001  # Tốc độ học
weight_decay = 0.0001  # Tham số weight decay

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Kiểm tra xem có GPU hay không
print(f"Using device: {device}")

# Khởi tạo mô hình, hàm mất mát và bộ tối ưu hóa
model = load_model(model_name + "/meta_model.pth", len(models_info), device).to(device)  # Tải mô hình từ file

optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
print("Initialization complete!")

# Lưu thông tin các tham số vào file
with open(model_name + "/meta_model.info", 'w') as f:
    f.write(f"Hyperparameters:\n")
    f.write(f"Max episodes: {max_episodes}\n")
    f.write(f"Batch size: {batch_size}\n")
    f.write(f"Number of samples: {len(training_data)}\n")
    f.write(f"Learning rate: {learning_rate}\n")
    f.write(f"Weight decay: {weight_decay}\n")
    f.write(f"Device: {device}\n")

    f.write(f"\nModel configuration:\n")
    f.write(f"Number of classes: {num_classes}\n")

print("Model configuration saved to " + model_name + "/meta_model.info")

Using device: cuda
Model loaded from meta_model01/meta_model.pth
Initialization complete!
Model configuration saved to meta_model01/meta_model.info


In [51]:
# Vòng lặp huấn luyện
for episode in range(max_episodes):
    #1. Lấy một batch dữ liệu ngẫu nhiên từ training_data
    batch = training_data.sample(batch_size)

    #2. Tách dữ liệu thành các phần riêng biệt
    agent_positions = torch.tensor([sample['agent_position'] for sample in batch], dtype=torch.float32)
    distances = torch.tensor([sample['distance'] for sample in batch], dtype=torch.float32)
    labels = torch.tensor([sample['label'] for sample in batch], dtype=torch.int)

    #3. Chuyển dữ liệu sang GPU nếu có
    agent_positions = agent_positions.to(device)
    distances = distances.to(device)
    labels = labels.to(device)

    #4. Đưa dữ liệu vào mô hình và tính toán dự đoán
    predictions = model(agent_positions)
    prediction = predictions[torch.arange(batch_size), labels - 1]

    #5. Tính toán hàm mất mát
    loss = F.mse_loss(prediction, distances)
    
    #6. Tối ưu hóa mô hình
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    #7. In thông tin về quá trình huấn luyện
    print(f"Episode {episode + 1}/{max_episodes}, Loss: {loss.item()}")


Episode 1/50000, Loss: 9.227395057678223
Episode 2/50000, Loss: 5.957434177398682
Episode 3/50000, Loss: 6.135538101196289
Episode 4/50000, Loss: 10.274898529052734
Episode 5/50000, Loss: 9.859542846679688
Episode 6/50000, Loss: 8.842192649841309
Episode 7/50000, Loss: 7.388291358947754
Episode 8/50000, Loss: 4.8610429763793945
Episode 9/50000, Loss: 6.933995246887207
Episode 10/50000, Loss: 6.232938766479492
Episode 11/50000, Loss: 2.9514994621276855
Episode 12/50000, Loss: 9.510854721069336
Episode 13/50000, Loss: 12.794846534729004
Episode 14/50000, Loss: 8.577250480651855
Episode 15/50000, Loss: 14.60760498046875
Episode 16/50000, Loss: 12.545777320861816
Episode 17/50000, Loss: 7.888154029846191
Episode 18/50000, Loss: 5.350531101226807
Episode 19/50000, Loss: 4.585252285003662
Episode 20/50000, Loss: 7.495032787322998
Episode 21/50000, Loss: 5.444854736328125
Episode 22/50000, Loss: 2.978119373321533
Episode 23/50000, Loss: 2.8756375312805176
Episode 24/50000, Loss: 6.42248535156

In [52]:
# Lưu trọng số của mô hình
save_model(model, model_name + '/meta_model.pth')

Model saved to meta_model01/meta_model.pth
